In [15]:
import pandas as pd
from sklearn.metrics import *
import numpy as np
import plotly.express as px
TRUE_LABELS = np.array([0, 1, 1, 0, 0, 1, 1, 1, 0, 0])
GROUP_MAP = {
    1: 'Correct AI',
    2: 'Random AI',
    3: 'Control (No AI)'
}

In [16]:
df = pd.read_csv('results.csv')
df.head()

,question,answer,time,subject,group
0,1,0,72.0,1,1
1,2,1,29.0,1,1
2,3,1,48.0,1,1
3,4,0,48.0,1,1
4,5,0,46.0,1,1


# Analyzing Performance on Accurate Summarization

In [21]:
plot_dict = {k: [] for k in ('Experimental Group', 'Test Subject', 'Accuracy')}
for group in reversed(sorted(df['group'].unique())):
    group_df = df[df['group'] == group]
    for subject in group_df['subject'].unique():
        subject_df = group_df[group_df['subject'] == subject]
        subject_acc = accuracy_score(TRUE_LABELS, subject_df['answer'])
        plot_dict['Experimental Group'].append(GROUP_MAP[group])
        plot_dict['Test Subject'].append(str(subject))
        plot_dict['Accuracy'].append(subject_acc)
plot_df = pd.DataFrame(plot_dict)
fig = px.bar(
    plot_df,
    x='Experimental Group',
    y='Accuracy',
    color='Test Subject',
    barmode='group',
    range_y=[0,1]
)
fig.show()

# Analyzing Time Taken to get Answers

In [70]:
plot_df = df.drop(['answer', 'subject'], axis=1).groupby(['question', 'group'])
plot_df = pd.merge(plot_df.mean().rename({'time': 'Mean Time (s)'}, axis=1), plot_df.std().rename({'time': 'std_time'}, axis=1), how='left', on=['question', 'group'])
plot_df = plot_df.reset_index()
plot_df['Experimental Group'] = plot_df['group'].apply(lambda x: GROUP_MAP[x])
plot_df['Question Number'] = plot_df['question'].apply(lambda x: str(x))
fig = px.bar(
    plot_df,
    x='Question Number',
    y='Mean Time (s)',
    color='Experimental Group',
    barmode='group',
    error_y='std_time'
)
fig.show()